In [327]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt
%inline matplotlib


UsageError: Line magic function `%inline` not found.


In [328]:
#We used a (bad) model to generate a large list of names. now we will try to build a classifier to see if we can tell the difference between real and fake names.
real_names = open('names.txt', 'r').read().splitlines()
fake_names = open('generated_names.txt', 'r').read().splitlines()
len(real_names), len(fake_names)
#see if theere are names in both lists
for name in real_names:
    if name in fake_names:
        print(name)
        break


emma


In [329]:
chars = sorted(list(set(''.join(real_names+fake_names))))
stoi = {c:i+1 for i,c in enumerate(chars)}
stoi['.'] = 0
itos = {i:c for c,i in stoi.items()}


In [330]:
#find longest name
maxlen = max([len(n) for n in real_names+fake_names])
maxlen

#pad all names to maxlen
real_names = [n.ljust(maxlen, '.') for n in real_names]
fake_names = [n.ljust(maxlen, '.') for n in fake_names]

#print first 10 names
real_names[:10]




['emma................',
 'olivia..............',
 'ava.................',
 'isabella............',
 'sophia..............',
 'charlotte...........',
 'mia.................',
 'amelia..............',
 'harper..............',
 'evelyn..............']

In [331]:
# create Y's
Y = torch.cat([torch.ones(len(real_names)), torch.zeros(len(fake_names))])
Y.shape
#add a dimension to Y
Y = Y.unsqueeze(1)


# create X's
X = torch.cat([torch.LongTensor([[stoi[c] for c in n]]) for n in real_names+fake_names])
X.shape





torch.Size([64066, 20])

In [332]:
embedding_dim = 24

W1 = torch.randn(X.shape[1]*embedding_dim, 750, requires_grad=True)
b1 = torch.randn(750, requires_grad=True)
W2 = torch.randn(750, 1, requires_grad=True)
b2 = torch.randn(1, requires_grad=True)

C = torch.randn((27, embedding_dim), requires_grad=True)
params = [W1, b1, W2, b2, C] 


def accuracy(pred, Y):
    return (pred.round() == Y).float().mean()


lr = 0.01




In [333]:
batch_size = 64
for i in range(100000):
    ix = torch.randint(0, X.shape[0], (batch_size,))
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 20*embedding_dim) @ W1 + b1)
    logits = h @ W2 + b2
    #we use binary cross entropy loss because we are doing binary classification
    loss = F.binary_cross_entropy_with_logits(logits, Y[ix])
    loss.backward()
    acc = accuracy(logits, Y[ix])
    for p in params:
        p.data -= p.grad * lr
        p.grad.zero_()
    if i % 100 == 0:
        print(f'loss: {loss.data:.3f}')

loss: 8.288
loss: 3.827
loss: 2.888
loss: 3.814
loss: 2.999
loss: 3.292
loss: 1.690
loss: 3.071
loss: 2.188
loss: 1.939
loss: 2.356
loss: 1.879
loss: 1.663
loss: 1.705
loss: 2.179
loss: 1.592
loss: 1.150
loss: 1.924
loss: 2.189
loss: 1.641
loss: 1.466
loss: 1.791
loss: 1.515
loss: 1.396
loss: 1.490
loss: 1.081
loss: 1.332
loss: 1.263
loss: 1.138
loss: 0.946
loss: 1.321
loss: 1.037
loss: 1.063
loss: 1.129
loss: 1.429
loss: 1.205
loss: 1.069
loss: 1.182
loss: 0.595
loss: 0.924
loss: 1.002
loss: 1.781
loss: 1.281
loss: 1.305
loss: 1.165
loss: 1.263
loss: 1.077
loss: 0.958
loss: 0.874
loss: 0.756
loss: 0.985
loss: 0.865
loss: 1.270
loss: 0.985
loss: 0.941
loss: 0.455
loss: 0.712
loss: 1.009
loss: 0.730
loss: 0.813
loss: 0.775
loss: 0.714
loss: 0.615
loss: 0.851
loss: 0.636
loss: 0.658
loss: 1.030
loss: 0.606
loss: 0.735
loss: 1.081
loss: 0.859
loss: 0.798
loss: 0.691
loss: 0.895
loss: 0.672
loss: 0.492
loss: 1.092
loss: 0.752
loss: 0.684
loss: 0.950
loss: 0.867
loss: 0.666
loss: 0.504
loss

In [ ]:
#accuracy on whole dataset
pred = torch.sigmoid((torch.tanh(C[X].view(-1, 20*embedding_dim) @ W1 + b1) @ W2 + b2))
accuracy(pred, Y)



tensor(0.7734)